In [1]:
import pandas as pd
from math import floor
from datetime import datetime
from datetime import timedelta

In [2]:
df =  pd.read_csv(r'../Data/dericavao_24h.csv')
intervalo = 10
tma = 156

In [3]:
def get_dataframe_sla():
    # Criar lista de horários com intervalo especificado
    horarios = []
    for hora in range(24):
        for minuto in range(0, 60, intervalo):
            horarios.append(f"{hora:02d}:{minuto:02d}")
        
        # Adicionar último horário se não completar 60 minutos
        if 60 % intervalo != 0 and (60 // intervalo) * intervalo != 60:
            horarios.append(f"{hora:02d}:59")

    # Criar DataFrame base
    df_intervalo = pd.DataFrame({'horario': horarios})

    # Extrair hora base (formato HH:00)
    df_intervalo['hora_base'] = df_intervalo['horario'].str[:2] + ':00'

    # Mesclar com dados originais
    df_merged = pd.merge(df_intervalo, df, left_on='hora_base', right_on='horario', suffixes=('', '_hora'))

    # Calcular proporção
    if 60 % intervalo == 0:
        # Divisão uniforme
        proporcao = intervalo / 60
        df_merged['quantidade'] = df_merged['quantidade'] * proporcao
    else:
        # Divisão não uniforme - distribuição igualitária
        intervalos_por_hora = df_merged.groupby('hora_base')['horario'].transform('count')
        df_merged['quantidade'] = df_merged['quantidade'] / intervalos_por_hora

    # Selecionar colunas finais
    df_final = df_merged[['horario', 'quantidade']]
    return df_final

In [4]:
df_final = get_dataframe_sla()

In [5]:
def encontrar_indice_por_hora_aproximada(df, horario_alvo: str):
    """
    Encontra o índice do horário mais próximo dentro da mesma hora.
    """

    alvo = datetime.strptime(horario_alvo, "%H:%M")
    alvo_hora = alvo.hour

    df = df.copy()  # Evita modificar o DataFrame original
    df['__hora'] = df['horario'].apply(lambda x: int(x.split(":")[0]))
    df['__minuto'] = df['horario'].apply(lambda x: int(x.split(":")[1]))

    candidatos = df[df['__hora'] == alvo_hora].copy()

    if candidatos.empty:
        raise ValueError(f"Nenhum horário encontrado para a hora {alvo_hora:02d}")

    candidatos['diff'] = candidatos['__minuto'].apply(lambda m: abs(m - alvo.minute))
    resultado = candidatos.sort_values(by='diff').iloc[0]

    return resultado.name

In [6]:
def str_to_time(hora_str):
    return datetime.strptime(hora_str, "%H:%M")

def set_capacidade_producao():
    df_sla = df_final.reset_index(drop=True)
   
    entrada = "07:30"
    almoco_inicio = "12:40"
    almoco_duracao = 60  # minutos
    saida = "15:20"

    entrada_dt = str_to_time(entrada)
    almoco_inicio_dt = str_to_time(almoco_inicio)
    almoco_fim_dt = almoco_inicio_dt + timedelta(minutes=almoco_duracao)
    saida_dt = str_to_time(saida)

    tempo_trabalhado = []
    
    for i in range(len(df_sla)):
        hora_inicio = str_to_time(df_sla.loc[i, 'horario'])
        if i < len(df_sla) - 1:
            hora_fim = str_to_time(df_sla.loc[i + 1, 'horario'])
        else:
            hora_fim = hora_inicio + timedelta(minutes=intervalo)  # fallback

        minutos_trabalhados = 0

        # Interseção com jornada total
        trabalho_inicio = max(hora_inicio, entrada_dt)
        trabalho_fim = min(hora_fim, saida_dt)

        if trabalho_fim > trabalho_inicio:
            trabalho_total = trabalho_fim - trabalho_inicio

            # Subtrai interseção com o almoço
            pausa_inicio = max(trabalho_inicio, almoco_inicio_dt)
            pausa_fim = min(trabalho_fim, almoco_fim_dt)

            if pausa_fim > pausa_inicio:
                pausa = pausa_fim - pausa_inicio
            else:
                pausa = timedelta(minutes=0)

            minutos_validos = int((trabalho_total - pausa).seconds / 60)
            minutos_trabalhados = min(minutos_validos, int((hora_fim - hora_inicio).seconds / 60))  # ⚠️ limita

        # print(f"H:{df_sla.loc[i, 'horario']} MT:{minutos_trabalhados}")
        tempo_trabalhado.append(minutos_trabalhados)
                
    # Agora calcula capacidade
    capacidade = [floor(mins * 60 / tma) for mins in tempo_trabalhado]

    print(f"Tempo Trabalhado        {tempo_trabalhado}")
    print(f"Len                     {len(tempo_trabalhado)}")
    print(f"Capacidade              {capacidade}")
    print(f"Len                     {len(capacidade)}")

In [7]:
set_capacidade_producao()

Tempo Trabalhado        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 0, 0, 0, 0, 0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Len                     144
Capacidade              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Len              